# In-Silico Saturation Mutagenesis

How to perform saturation mutagenesis - systematically mutating every position in a genomic region to all possible nucleotides. This technique is powerful for:

- Identifying functional elements - Find which positions affect predictions
- Quantifying position importance - Measure sensitivity to mutations
- Predicting variant effects - Estimate impact of any possible SNV
- Finding regulatory motifs - Discover sequence patterns that matter

## Learning Objectives

- Generate all possible single-nucleotide mutations in a region
- Run model predictions on mutated sequences
- Visualize mutation effect landscapes
- Identify high-impact positions
- Compare regional vs. targeted mutagenesis approaches

## Setup

In [ ]:
import supremo_lite as sl
from supremo_lite.mock_models import TestModel, TORCH_AVAILABLE
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyfaidx import Fasta
import pandas as pd
import os

sns.set_style("whitegrid")

if not TORCH_AVAILABLE:
    raise ImportError("PyTorch required. Install with: pip install torch")

print(f"supremo_lite version: {sl.__version__}")

# Load reference
test_data_dir = "../../tests/data"
reference = Fasta(os.path.join(test_data_dir, "test_genome.fa"))

print(f"Reference loaded: {list(reference.keys())}")

## Part 1: Regional Saturation Mutagenesis

First, let's mutate every position in a genomic region using `get_sm_sequences()`.

In [ ]:
# Define a region to mutate
chrom = 'chr1'
start = 20
end = 50  # 30 bp region

# Generate all mutations
ref_seq, alt_seqs, metadata = sl.get_sm_sequences(
    chrom=chrom,
    start=start,
    end=end,
    reference_fasta=reference
)

print(f"Saturation mutagenesis results:")
print(f"  Region: {chrom}:{start}-{end} ({end-start} bp)")
print(f"  Reference sequence shape: {ref_seq.shape}")
print(f"  Alternative sequences shape: {alt_seqs.shape}")
print(f"  Number of mutations: {len(metadata)}")

# Show metadata
print(f"\nMetadata (first 10 mutations):")
print(metadata.head(10))

# Calculate expected number of mutations
region_len = end - start
expected_mutations = region_len * 3  # 3 alternative nucleotides per position
print(f"\nExpected mutations: {expected_mutations} (3 alternatives × {region_len} positions)")
print(f"Generated mutations: {len(metadata)}")
print(f"Match: {expected_mutations == len(metadata)}")

## Running Predictions on Mutated Sequences

In [ ]:
# Initialize model
model = TestModel(
    n_targets=1,  # Single signal for simplicity
    bin_size=1,   # Per-base resolution (no binning)
    crop_length=0  # No cropping
)

print(f"Model configuration:")
print(f"  Targets: {model.n_targets}")
print(f"  Bin size: {model.bin_size} (per-base predictions)")
print(f"  Crop length: {model.crop_length}")

# Get predictions
# Add batch dimension if needed
ref_seq_batch = ref_seq.unsqueeze(0) if len(ref_seq.shape) == 2 else ref_seq

ref_pred = model(ref_seq_batch)
alt_preds = model(alt_seqs)

print(f"\nPrediction shapes:")
print(f"  Reference: {ref_pred.shape}")
print(f"  Alternatives: {alt_preds.shape}")
print(f"  Format: (batch, targets, positions)")

## Calculating Mutation Effects

In [ ]:
# Calculate effect at mutation position for each variant
ref_pred_np = ref_pred.cpu().numpy() if hasattr(ref_pred, 'cpu') else ref_pred
alt_preds_np = alt_preds.cpu().numpy() if hasattr(alt_preds, 'cpu') else alt_preds

# For each mutation, get the prediction at the mutated position
effects = []
for i, row in metadata.iterrows():
    offset = row['offset']  # Position within the region (0-based)
    
    # Get reference and alternate predictions at this position
    ref_val = ref_pred_np[0, 0, offset]  # [batch=0, target=0, position=offset]
    alt_val = alt_preds_np[i, 0, offset]  # [mutation_i, target=0, position=offset]
    
    effect = alt_val - ref_val
    effects.append({
        'position': row['start'] + offset,  # Genomic position
        'offset': offset,
        'ref': row['ref'],
        'alt': row['alt'],
        'ref_pred': ref_val,
        'alt_pred': alt_val,
        'effect': effect
    })

effects_df = pd.DataFrame(effects)
print("Mutation effects (first 15):")
print(effects_df.head(15))

# Summary statistics
print(f"\nEffect statistics:")
print(f"  Mean absolute effect: {effects_df['effect'].abs().mean():.4f}")
print(f"  Max positive effect: {effects_df['effect'].max():.4f}")
print(f"  Max negative effect: {effects_df['effect'].min():.4f}")

## Visualizing the Mutation Effect Landscape

Create a heatmap showing the effect of each possible mutation:

In [ ]:
print("\nInterpretation:")
print("• Red: Mutations that increase prediction")
print("• Blue: Mutations that decrease prediction")
print("• White: Little to no effect")
print("• Green boxes: Reference nucleotide (no mutation)")
print("• Columns with strong colors: High-impact positions")

## Identifying High-Impact Positions

In [ ]:
# Calculate max absolute effect per position
position_importance = effects_df.groupby('position')['effect'].apply(
    lambda x: x.abs().max()
).reset_index()
position_importance.columns = ['position', 'max_abs_effect']
position_importance = position_importance.sort_values('max_abs_effect', ascending=False)

print("Top 10 most important positions:")
print(position_importance.head(10))

# Visualize position importance
plt.figure(figsize=(14, 5))
plt.bar(range(len(position_importance)), position_importance['max_abs_effect'].values,
        color='steelblue', edgecolor='black', linewidth=0.5)
plt.xlabel('Position (sorted by importance)', fontsize=11)
plt.ylabel('Max Absolute Effect', fontsize=11)
plt.title('Position Importance: Maximum Mutation Effect per Position', 
          fontsize=13, fontweight='bold')
plt.axhline(y=position_importance['max_abs_effect'].median(), 
            color='red', linestyle='--', label=f'Median = {position_importance["max_abs_effect"].median():.3f}')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Identify critical positions (> 75th percentile)
threshold = position_importance['max_abs_effect'].quantile(0.75)
critical_positions = position_importance[position_importance['max_abs_effect'] > threshold]

print(f"\nCritical positions (effect > {threshold:.3f}):")
for _, row in critical_positions.iterrows():
    pos = int(row['position'])
    offset = pos - start
    ref_nt = ref_sequence[offset]
    print(f"  Position {pos}: Reference = {ref_nt}, Max effect = {row['max_abs_effect']:.3f}")

## Part 2: Targeted Saturation Mutagenesis

For focused analysis, use `get_sm_subsequences()` to mutate only around a specific anchor point:

In [ ]:
# Define anchor point (e.g., a known regulatory site)
anchor = 35  # Center of our region
anchor_radius = 5  # Mutate ±5 bp around anchor
seq_len = 30  # Total sequence length for context

# Generate targeted mutations
ref_seq_targeted, alt_seqs_targeted, metadata_targeted = sl.get_sm_subsequences(
    chrom=chrom,
    anchor=anchor,
    anchor_radius=anchor_radius,
    seq_len=seq_len,
    reference_fasta=reference
)

print(f"Targeted saturation mutagenesis:")
print(f"  Anchor position: {chrom}:{anchor}")
print(f"  Mutation radius: ±{anchor_radius} bp")
print(f"  Sequence length: {seq_len} bp")
print(f"  Number of mutations: {len(metadata_targeted)}")

# Expected: (2 * radius + 1) * 3 mutations
expected = (2 * anchor_radius + 1) * 3
print(f"  Expected mutations: {expected}")

print(f"\nTargeted metadata:")
print(metadata_targeted.head(15))

## Comparing Regional vs. Targeted Approaches

In [ ]:
comparison = pd.DataFrame([
    {
        'Approach': 'Regional (get_sm_sequences)',
        'Function': 'get_sm_sequences()',
        'Region': f'{end-start} bp',
        'Mutations': len(metadata),
        'Use Case': 'Comprehensive screen of entire region',
        'Computational Cost': 'Higher',
        'Example': 'Find all functional elements in a promoter'
    },
    {
        'Approach': 'Targeted (get_sm_subsequences)',
        'Function': 'get_sm_subsequences()',
        'Region': f'±{anchor_radius} bp around anchor',
        'Mutations': len(metadata_targeted),
        'Use Case': 'Focused analysis of known site',
        'Computational Cost': 'Lower',
        'Example': 'Validate importance of TF binding site'
    }
])

print("\nApproach Comparison:")
print("=" * 100)
for _, row in comparison.iterrows():
    print(f"\n{row['Approach']}:")
    print(f"  Function: {row['Function']}")
    print(f"  Region: {row['Region']}")
    print(f"  Mutations: {row['Mutations']}")
    print(f"  Use case: {row['Use Case']}")
    print(f"  Cost: {row['Computational Cost']}")
    print(f"  Example: {row['Example']}")

## Practical Application: Finding Regulatory Motifs

Let's use the mutation effect data to identify potential regulatory motifs:

In [ ]:
# Find stretches of high-impact positions (potential motifs)
def find_motif_regions(position_importance, window_size=3, threshold_percentile=75):
    """Find consecutive high-impact positions (potential motifs)."""
    threshold = position_importance['max_abs_effect'].quantile(threshold_percentile / 100)
    
    # Sort by position
    sorted_pos = position_importance.sort_values('position')
    
    motifs = []
    current_motif = []
    
    for _, row in sorted_pos.iterrows():
        if row['max_abs_effect'] > threshold:
            if not current_motif or row['position'] == current_motif[-1]['position'] + 1:
                current_motif.append({'position': int(row['position']), 
                                     'effect': row['max_abs_effect']})
            else:
                if len(current_motif) >= window_size:
                    motifs.append(current_motif)
                current_motif = [{'position': int(row['position']), 
                                'effect': row['max_abs_effect']}]
        else:
            if len(current_motif) >= window_size:
                motifs.append(current_motif)
            current_motif = []
    
    if len(current_motif) >= window_size:
        motifs.append(current_motif)
    
    return motifs

motifs = find_motif_regions(position_importance, window_size=3, threshold_percentile=75)

print(f"Potential regulatory motifs (≥3 consecutive high-impact positions):")
print("=" * 80)

if motifs:
    for i, motif in enumerate(motifs, 1):
        positions = [m['position'] for m in motif]
        effects = [m['effect'] for m in motif]
        
        # Get sequence
        motif_start = positions[0]
        motif_end = positions[-1] + 1
        motif_seq = ref_sequence[(motif_start-start):(motif_end-start)]
        
        print(f"\nMotif {i}:")
        print(f"  Position: {chrom}:{motif_start}-{motif_end}")
        print(f"  Length: {len(positions)} bp")
        print(f"  Sequence: {motif_seq}")
        print(f"  Mean effect: {np.mean(effects):.3f}")
        print(f"  Max effect: {np.max(effects):.3f}")
        print(f"  Interpretation: Positions where mutations have large effects = likely functional")
else:
    print("  No motifs found with current threshold")
    print("  Try adjusting threshold_percentile or window_size")

## Best Practices for Saturation Mutagenesis

### 1. **Choose the right approach**
```python
# For exploratory analysis - screen entire region
ref_seq, alt_seqs, metadata = sl.get_sm_sequences(
    chrom='chr1', start=1000, end=1100, reference_fasta=ref
)

# For hypothesis testing - focus on specific site
ref_seq, alt_seqs, metadata = sl.get_sm_subsequences(
    chrom='chr1', anchor=1050, anchor_radius=10, 
    seq_len=200, reference_fasta=ref
)
```

### 2. **Consider computational resources**
```python
# Number of mutations = region_length × 3
# For 100 bp region: 300 mutations
# For 1000 bp region: 3000 mutations!

# Use batching for large regions
batch_size = 100
for i in range(0, len(alt_seqs), batch_size):
    batch = alt_seqs[i:i+batch_size]
    preds = model(batch)
    # Process predictions...
```

### 3. **Interpret effects carefully**
```python
# Don't just look at magnitude - consider:
# 1. Direction of effect (increase vs decrease)
# 2. Context (neighboring positions)
# 3. Biological plausibility

# Calculate different effect metrics
effects['abs_effect'] = effects['effect'].abs()
effects['relative_effect'] = effects['effect'] / effects['ref_pred']
effects['direction'] = np.sign(effects['effect'])
```

### 4. **Validate findings**
```python
# Cross-validate with:
# - Multiple models
# - Different sequence lengths
# - Known functional elements
# - Experimental data (if available)
```

## Summary

In this notebook, you learned comprehensive saturation mutagenesis workflows:

1. Regional mutagenesis - `get_sm_sequences()` for complete region screening
2. Targeted mutagenesis - `get_sm_subsequences()` for focused analysis
3. Effect calculation - Measuring mutation impact on predictions
4. Visualization - Heatmaps and importance plots
5. Position ranking - Identifying high-impact positions
6. Motif finding - Discovering potential regulatory elements
7. Best practices - Guidelines for effective mutagenesis studies

## Use Cases

Saturation mutagenesis is powerful for:
- Variant interpretation: Predict effects of any possible SNV
- Regulatory element discovery: Find functional motifs
- Model interpretation: Understand what sequence features matter
- Therapeutic design: Identify critical positions for targeting
- CRISPR guide design: Avoid disrupting functional elements

## Documentation

For more details, see:
- **[User Guide: Saturation Mutagenesis](../user_guide/mutagenesis.md)** - Detailed mutagenesis documentation
- **[API Reference](../autoapi/index.rst)** - Function signatures and parameters